In [1]:
import os
op = os.path
opd = op.dirname
opj = op.join
orig_dir = os.getcwd()

In [2]:
os.chdir( opj( opd( opd( orig_dir ) ) ) )

In [3]:
%matplotlib
##
from glob import glob
from pathlib import Path
from time import sleep, time
import yaml
import re
import numpy as np
from matplotlib import pyplot as plt
import cv2
##
from modules.devices.Olympuslx81 import OLYMP
from modules.devices.Prior import PRIOR
from modules.devices.Evolve512 import EVOLVE
###
from tensorflow.keras import models
import tensorflow.keras as tfk

Using matplotlib backend: Qt5Agg


In [4]:
ol = OLYMP()
pr = PRIOR()
ev = EVOLVE()

########### For BF autocontrast is True


In [5]:
mod_fold = 'Z:/Lionel_Chiron/Projects/AFML/fast_AFML/models_sets/models_yeast_jan4th2023/*'
lm = glob(mod_fold)
# print(lm)
# models_dec4th2022

## Load the models

In [6]:
lm_sorted = sorted(lm, key= lambda elem : int(re.findall('lev(-?\\d+)', elem)[0]))
#print(f'lm_sorted { lm_sorted }')
list_mod = []
t0 = time()
print('loading the models for fast AFML.. ')
for m in lm_sorted:
    list_mod += [models.load_model(m)]
t1 = time()
print(f'time elapsed for loading the models is { round((t1-t0)/60,1) } min')

loading the models for fast AFML.. 
time elapsed for loading the models is 4.4 min


In [9]:
def take_pic_posz(i, val, bpp=8, move_type='d', show=False):
    '''
    take the picture at pos posz
    '''
    ol.go_zpos( val, move_type=move_type )            # 797300 
    answ = ol.receive(13)
    print(f'answ 0 {answ}')
    #addr_img = f'uu{i}.tiff'
    addr_img = f'bf_for_optimz{i}.tiff'
    ev.take_pic(addr=addr_img, bpp=bpp, exp_time=100)
#     ev.handle_contrast(True, bpp)
    ev.adapt(bpp)
    if show: plt.imshow(ev.frame, cmap='gray')
    return ev.frame

In [10]:
def make_pred(i, mod=None, thresh=127, show=False):
    '''
    make the prediction and return surface 
    '''
    img = cv2.imread(f'bf_for_optimz{i}.tiff')
    arr = np.array([ img ], dtype=np.float32)/255                        # format array for prediction
    pred = mod.predict(arr)[0]*255
    ##
    ll = len( np.where( pred > thresh )[0] )
    #print(f'll { ll }')
    addr_pred = f'pred_uu{i}.png'
    cv2.imwrite(addr_pred, pred)
    if show:
        plt.figure()
        plt.imshow(pred, cmap='gray')
    return ll

In [14]:
def find_z_optim(z, step=20, thr=30, fill_dic_surf=False):
    '''
    find the optimal focus
    z: z position eg: 785400
    '''
    t0 = time()
    
    pic0 = take_pic_posz(0, z , bpp=8, move_type='d')
    pic1 = take_pic_posz(1, z + step, bpp=8, move_type='d')
    ldiff = []
    print('--------')
    optim = 0
    max_surf = 0
    # Applying the models.. 
    for num, m in enumerate(list_mod):
        surf0 = make_pred(0, mod=m, thresh=thr, show=False)
        surf1 = make_pred(1, mod=m, thresh=thr, show=False)
        diff = surf1-surf0
        if max_surf < surf0:
            max_surf = surf0
        ldiff += [[surf0, diff]]
        print(f'for mod {num}, diff is {diff}')
        if fill_dic_surf:
            try:
                dic_surf[num] += [surf0]
            except:
                dic_surf[num] = [surf0]
    print(f'max_surf = { max_surf }')
    for mind,(surf,slope) in enumerate(ldiff):
        if surf > max_surf/5 and mind>0:
            if np.sign(slope) == -np.sign(old_slope):
                if np.abs(slope)< np.abs(old_slope):
                    optim = mind
                else:
                    optim = mind-1
                break
        old_slope = slope
            
    
    print(f'***** optim is {optim} *****')
    t1 = time()
    print(f'time elapsed is { round((t1-t0),1) } s')
    return optim

In [12]:
def plot_curves(lcurves, mod_mid):
    '''
    '''
    lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step = lcurves
    plt.plot(lposz, label='input')
    plt.plot(lposz_corr, label='correction')
    plt.legend()
    print('finished..')
    date = 'jan4th2022'
    plt.savefig(f'meth_13mod_stability_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel('optim')
    plt.plot(loptim)
    plt.savefig(f'list_optim_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel(f'surf model {mod_mid}')
    plt.plot(lsurf0)
    plt.savefig(f'list_mod0_zmid{z}_step{step}_{date}.png')
    plt.figure()
    plt.ylabel(f'diff model {mod_mid}')
    plt.plot(ldiff2)
    plt.savefig(f'list_ldiff2_zmid{z}_step{step}_{date}.png')

In [11]:
def test_focus_conservation(z,
                            nb_steps=10,
                            step=20,
                            thr=30,
                            mod_mid=9,
                            delta_mod=30   # dist from one model to the other.. 
                            ):
    '''
    '''
    lposz = []
    lposz_corr = []
    loptim = []
    lsurf0 = []
    ldiff2 = []
    
    
    for j in range(nb_steps):
        print(f'######################## j is {j}')
        currz = z + (j-nb_steps//2)*step
        for i in range(3):
            ol.go_zpos( currz, move_type='d' )
        optim = find_z_optim(currz)
        loptim += [optim]
        ref = (len(list_mod)-1)//2
        posz_corr = currz-(optim-ref)*delta_mod #+ 2*(10-optim)
        print(f'##################### posz_corr = {posz_corr}')
        for i in range(3):
            ol.go_zpos( posz_corr, move_type='d' )
        take_pic_posz(3+j, posz_corr, bpp=8, move_type='d')
        lposz += [currz]
        lposz_corr += [posz_corr]
#         num_img = (3+j)+100
#         addr_img = f'bf_for_optimz{num_img}.tiff'
#         ev.take_pic(addr=addr_img, bpp=8, exp_time=100)
        surf0 = make_pred(3+j, mod=list_mod[mod_mid], thresh=thr, show=False)
        lsurf0 +=[surf0]
        take_pic_posz(3+j+100, posz_corr + step, bpp=8, move_type='d')
        surf1 = make_pred(3+j+100, mod=list_mod[mod_mid], thresh=thr, show=False)
        diff = surf1-surf0
        ldiff2 += [diff]
    lcurves = [lposz, lposz_corr, loptim, lsurf0, ldiff2, z, step]
    plot_curves(lcurves, mod_mid)

In [17]:
test_focus_conservation(725395,
                        nb_steps=30,
                        step=30,
                        thr=30,
                        mod_mid=9)

######################## j is 0
In go_zpos, val = 724945 
In go_zpos, val = 724945 
In go_zpos, val = 724945 
In go_zpos, val = 724945 
answ 0 OV !,E02110

apply the autocontrast..
In go_zpos, val = 724965 
answ 0 2LOG !,E02511
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 0
for mod 7, diff is 0
for mod 8, diff is 0
for mod 9, diff is 0
for mod 10, diff is 0
for mod 11, diff is 0
for mod 12, diff is 0
for mod 13, diff is 0
for mod 14, diff is 0
for mod 15, diff is 0
for mod 16, diff is 0
max_surf = 0
***** optim is 0 *****
time elapsed is 4.0 s
##################### posz_corr = 725185
In go_zpos, val = 725185 
In go_zpos, val = 725185 
In go_zpos, val = 725185 
In go_zpos, val = 725185 
answ 0 
2MOV +
2LO
apply the autocontrast..
In go_zpos, val = 725215 
answ 0 G +
2MOV +

apply the autocontrast..
######################## j is 1
In go_zpos, val = 724975 

In go_zpos, val = 725425 
In go_zpos, val = 725425 
In go_zpos, val = 725425 
answ 0  +
2MOV !,E0
apply the autocontrast..
In go_zpos, val = 725455 
answ 0 2110
2LOG !,
apply the autocontrast..
######################## j is 9
In go_zpos, val = 725215 
In go_zpos, val = 725215 
In go_zpos, val = 725215 
In go_zpos, val = 725215 
answ 0 E02511
2LOG 
apply the autocontrast..
In go_zpos, val = 725235 
answ 0 +
2MOV !,E02
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 0
for mod 7, diff is 0
for mod 8, diff is 0
for mod 9, diff is 0
for mod 10, diff is 0
for mod 11, diff is 0
for mod 12, diff is 0
for mod 13, diff is 0
for mod 14, diff is 0
for mod 15, diff is 0
for mod 16, diff is 0
max_surf = 0
***** optim is 0 *****
time elapsed is 3.8 s
##################### posz_corr = 725455
In go_zpos, val = 725455 
In go_zpos, val = 725455 
In go_zpos, val = 725455 
In g

for mod 8, diff is 0
for mod 9, diff is 0
for mod 10, diff is 0
for mod 11, diff is 0
for mod 12, diff is 0
for mod 13, diff is 0
for mod 14, diff is 0
for mod 15, diff is 0
for mod 16, diff is 23
max_surf = 2453
***** optim is 0 *****
time elapsed is 3.8 s
##################### posz_corr = 725695
In go_zpos, val = 725695 
In go_zpos, val = 725695 
In go_zpos, val = 725695 
In go_zpos, val = 725695 
answ 0 2LOG +
2MOV 
apply the autocontrast..
In go_zpos, val = 725725 
answ 0 !,E02110
2LO
apply the autocontrast..
######################## j is 18
In go_zpos, val = 725485 
In go_zpos, val = 725485 
In go_zpos, val = 725485 
In go_zpos, val = 725485 
answ 0 G !,E02511
2
apply the autocontrast..
In go_zpos, val = 725505 
answ 0 LOG +
2MOV !
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 0
for mod 7, diff is 0
for mod 8, diff is 0
for mod 9, diff is 0
for mod 1

answ 0 12LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 725745 
answ 0  2LOG !,E0251
apply the autocontrast..
--------
for mod 0, diff is 0
for mod 1, diff is 0
for mod 2, diff is 0
for mod 3, diff is 0
for mod 4, diff is 0
for mod 5, diff is 0
for mod 6, diff is 0
for mod 7, diff is 0
for mod 8, diff is 0
for mod 9, diff is 0
for mod 10, diff is 0
for mod 11, diff is 0
for mod 12, diff is 0
for mod 13, diff is 0
for mod 14, diff is 0
for mod 15, diff is 0
for mod 16, diff is 84
max_surf = 1059
***** optim is 0 *****
time elapsed is 3.8 s
##################### posz_corr = 725965
In go_zpos, val = 725965 
In go_zpos, val = 725965 
In go_zpos, val = 725965 
In go_zpos, val = 725965 
answ 0 12LOG +
2LOG
apply the autocontrast..
In go_zpos, val = 725995 
answ 0  2LOG !,E0251
apply the autocontrast..
######################## j is 27
In go_zpos, val = 725755 
In go_zpos, val = 725755 
In go_zpos, val = 725755 
In go_zpos, val = 725755 
answ 0 12LOG +
2LOG
apply the autocontrast..
In g

## make the profiles with zstack

In [15]:
dic_surf = {}
loptim = []
nbstep = 40
step = 30
for i in range(0,nbstep):  # 40
    print(f'############################## i = {i}')
    optim = find_z_optim(725395+(i-nbstep//2)*step, 
                        step=30,
                        thr=30,
                        fill_dic_surf=True) # 833600
    loptim += [optim]
print('finished')

############################## i = 0
In go_zpos, val = 724795 
answ 0  +
2LOG +
2
apply the autocontrast..
In go_zpos, val = 724825 
answ 0 LOG +
2LOG !
apply the autocontrast..
--------
for mod 0, diff is -885
for mod 1, diff is 1423
for mod 2, diff is 2277
for mod 3, diff is 771
for mod 4, diff is 47
for mod 5, diff is -290
for mod 6, diff is -639
for mod 7, diff is -111
for mod 8, diff is -316
for mod 9, diff is -1070
for mod 10, diff is -854
for mod 11, diff is -548
for mod 12, diff is -683
for mod 13, diff is -351
for mod 14, diff is 0
for mod 15, diff is 0
for mod 16, diff is 201
max_surf = 29616
***** optim is 0 *****
time elapsed is 11.9 s
############################## i = 1
In go_zpos, val = 724825 
answ 0 ,E02511
2MOV
apply the autocontrast..
In go_zpos, val = 724855 
answ 0  +
2LOG +
2
apply the autocontrast..
--------
for mod 0, diff is -797
for mod 1, diff is -549
for mod 2, diff is 2643
for mod 3, diff is 1750
for mod 4, diff is 1532
for mod 5, diff is -195
for mod 6, di

for mod 8, diff is -4659
for mod 9, diff is -393
for mod 10, diff is 134
for mod 11, diff is 1206
for mod 12, diff is -931
for mod 13, diff is 3088
for mod 14, diff is 49
for mod 15, diff is 140
for mod 16, diff is 76
max_surf = 36930
***** optim is 3 *****
time elapsed is 3.6 s
############################## i = 13
In go_zpos, val = 725185 
answ 0 1
2MOV +
2L
apply the autocontrast..
In go_zpos, val = 725215 
answ 0 OG +
2MOV +
apply the autocontrast..
--------
for mod 0, diff is 799
for mod 1, diff is -27
for mod 2, diff is 1332
for mod 3, diff is 945
for mod 4, diff is 688
for mod 5, diff is 420
for mod 6, diff is -585
for mod 7, diff is -53
for mod 8, diff is -63
for mod 9, diff is 259
for mod 10, diff is 137
for mod 11, diff is 288
for mod 12, diff is 1502
for mod 13, diff is 1226
for mod 14, diff is 245
for mod 15, diff is 246
for mod 16, diff is 97
max_surf = 37451
***** optim is 1 *****
time elapsed is 3.7 s
############################## i = 14
In go_zpos, val = 725215 
answ 0

apply the autocontrast..
In go_zpos, val = 725575 
answ 0 E02511
2MOV 
apply the autocontrast..
--------
for mod 0, diff is -2679
for mod 1, diff is 1153
for mod 2, diff is 101
for mod 3, diff is -1525
for mod 4, diff is -2025
for mod 5, diff is -728
for mod 6, diff is -441
for mod 7, diff is -64
for mod 8, diff is -749
for mod 9, diff is -2540
for mod 10, diff is -898
for mod 11, diff is -395
for mod 12, diff is -3225
for mod 13, diff is -795
for mod 14, diff is -142
for mod 15, diff is -543
for mod 16, diff is -1869
max_surf = 42414
***** optim is 2 *****
time elapsed is 3.7 s
############################## i = 26
In go_zpos, val = 725575 
answ 0 +
2LOG +
2M
apply the autocontrast..
In go_zpos, val = 725605 
answ 0 OV +
2LOG +
apply the autocontrast..
--------
for mod 0, diff is -3160
for mod 1, diff is 419
for mod 2, diff is -275
for mod 3, diff is -1338
for mod 4, diff is -1838
for mod 5, diff is -547
for mod 6, diff is -322
for mod 7, diff is -177
for mod 8, diff is -521
for mod 9

for mod 5, diff is -81
for mod 6, diff is -52
for mod 7, diff is -49
for mod 8, diff is -32
for mod 9, diff is -827
for mod 10, diff is -166
for mod 11, diff is -137
for mod 12, diff is -661
for mod 13, diff is -608
for mod 14, diff is -2
for mod 15, diff is -200
for mod 16, diff is -1027
max_surf = 21561
***** optim is 0 *****
time elapsed is 3.7 s
############################## i = 38
In go_zpos, val = 725935 
answ 0 OG !,E02511

apply the autocontrast..
In go_zpos, val = 725965 
answ 0 2MOV +
2LOG 
apply the autocontrast..
--------
for mod 0, diff is -1081
for mod 1, diff is -450
for mod 2, diff is -1579
for mod 3, diff is -2181
for mod 4, diff is -110
for mod 5, diff is -15
for mod 6, diff is 24
for mod 7, diff is 0
for mod 8, diff is 38
for mod 9, diff is -534
for mod 10, diff is -113
for mod 11, diff is -32
for mod 12, diff is -324
for mod 13, diff is -617
for mod 14, diff is -2
for mod 15, diff is -135
for mod 16, diff is -963
max_surf = 17940
***** optim is 8 *****
time elapsed

In [ ]:
dir(ol)

In [ ]:
ev.retrieve_autocontrast()

## plot the profiles and predictions.. 

In [16]:
allmax = 0
for k,v in dic_surf.items():
    print(k)
    maxsurf = max(dic_surf[k])
    if allmax<maxsurf:
        allmax=maxsurf
fact_norm = (len(list_mod)-1)/allmax
for mind, (k,v) in enumerate(dic_surf.items()):
    print(k)
    plt.plot(np.array(dic_surf[k])*fact_norm, alpha=0.3, label=f'lev{mind}')
# for pt in loptim:
#     plt.plot(pt,'rx')
plt.legend()
loptim.insert(0,0)
plt.plot(loptim,'rx')
    
plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


##  Controls

In [ ]:
print(ol.ask_zpos())

In [ ]:
for k,v in dic_surf.items():
    print(k)
    if k!='max':
        plt.plot(np.array(dic_surf[k]))

In [ ]:
find_z_optim(834570) # 833812

In [ ]:
curr_pos = 830250
for i in range(3):
    ol.go_zpos( curr_pos, move_type='d' )

In [ ]:
find_z_optim(curr_pos)

In [ ]:
for i in range(3):
    take_pic_posz(301, curr_pos-10 , bpp=8, move_type='d')

In [ ]:
def aa(ll):
    a, b, c = ll
    print(a,b,c)
aa([1,6,7])

In [ ]:
dir(ol)

In [ ]:
ol.set_shutter('on')